# Python 3.10: Cool New Features for You to Try

The code for this notebook is from this [RealPython article](https://realpython.com/python310-new-features/)

In [ ]:
import datetime

In [ ]:
import pendulum
import requests  # Used to access random user data from `https://randomuser.me/`

## Better Error Messages

Run the examples:

- `rp310_hello.py`
- `rp310_unterminated_dict.py`
- `rp310_missing_comma.py`
- `rp310_assignment_equality.py`
- `rp310_misspell_1.py`
- `rp310_misspell_2.py`
- `rp310_misspell_3.py`

## Structural Pattern Matching

We'll demonstrate this feature with three different examples:
- Detecting and deconstructing different **structures** in your data
- Using different kinds of **patterns**
- **Matching** literal patterns

We'll also include links for more details.

### Deconstructing Data Structures

In [ ]:
a_user = {
    'name': {'first': 'Pablo', 'last': 'Galindo Salgado'},
    'title': 'Python 3.10 release manager',
}

match a_user:
    case {'name': {'first': first_name}}:
        pass

# noinspection PyUnboundLocalVariable
first_name

#### Getting Random User Data

Using `requests` to obtain different versions of the user data using the API.

In [ ]:
def get_user(version='1.3'):
    """Get random user information."""
    raw_url = f'https://randomuser.me/api/{version}/?result=1'
    # Typically, one would use the `raw_url` defined above; however, we want
    # to duplicate the random data from the
    # [RealPython article](https://realpython.com/python310-new-features/).
    # Therefore, we add a `seed` parameter to the URL with the value 310.
    #
    # Note that the JSON object returned by the API contains metadata that
    # includes **both** the version of the data as well as the seed used to
    # create the random user.
    url = f'{raw_url}&seed=310'
    response = requests.get(url)
    if response:
        return response.json()['results'][0]

In [ ]:
user_13 = get_user()
user_13

Compare the previous result with a version 1.1 random user:

In [ ]:
user_11 = get_user(version='1.1')
user_11

One of the members changed between version 1.1 and version 1.3 is `dob`. In version 1.3, the value for `dob` is a dictionary with two keys: 'data' and 'name'.


In [ ]:
user_13['dob']

In version 1.1, the value of `dob` is of type `string` containing the date of birth. As a result, the developer must **calculate** the age.

Additionally, remember that the `age` returned by the 1.3 API is accurate when the data is returned. If you store this data, this value will eventually become outdated. If this is a concern, one should calculate the current age based on the `date` field.

In [ ]:
user_11['dob']

Before Python 3.10, to calculate the age from **different** API versions, one would use an `if` statement and perform different calculations based on the API version.

In Python 3.10, we can use structural pattern matching instead!

In [ ]:
def get_age(user):
    """Get the age of a user."""
    match user:
        # Note that patterns are matched **in order**; that as the 1.3
        # version of `dob` will be tried **before** matching the 1.1 version.
        # If we reversed the order of the two `case` clauses (patterns),
        # Python would **never** match the 1.3 pattern because it would
        # always match the 1.1 version.
        #
        # The moral of the story, order candidate patterns from most
        # specific to most general.
        case {'dob': {'age': int(age)}}:
            return age
        case {'dob': dob_text}:
            print(f'{dob_text=}')
            dob = pendulum.parse(dob_text)
            now = pendulum.now(tz=dob.tz)
            return (now - dob).in_years()

In [ ]:
user_13['dob'], get_age(user_13)

In [ ]:
user_11['dob'], get_age(user_11)